In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import lxml.etree as ET
from xml.dom import minidom
import pathlib
import glob
import datetime
import sys

In [21]:
def makeprevurl(recordurl,filename):
    return recordurl+"/preview/"+filename

def getfiles(prevurl):
    files = []

    try :
        r = requests.get(prevurl.replace('https','http'))
    except :
        r=""    
    prev=BeautifulSoup(r.text, 'html.parser')
    for t in prev.select(".fa-file-o"):
        k=t.find_previous("li").span
        files.append({'filename':k.contents[1].encode(encoding='ascii',errors='ignore').decode(),
                        'extension':pathlib.Path(k.contents[1]).suffix.encode(encoding='ascii',errors='ignore').decode(),
                        'size':k.next_sibling.next_sibling.contents[0].encode(encoding='ascii',errors='ignore').decode()})

    return files

def getarchives(recordurl):
    print(recordurl)
    files = []
    try :
        r = requests.get(recordurl.replace('https','http'))
    except :
             r=""
    page=BeautifulSoup(r.text, 'html.parser')
    print(page)
    for t in page.select(".filename"):
        k=t.find_previous("td")
        filename=k.a.contents[0]
        ext=pathlib.Path(filename).suffix
        if ext==".zip":
            archive=pd.DataFrame.from_dict(getfiles(makeprevurl(recordurl,filename)))
        else :
            archive=pd.DataFrame(columns=['filename','extension','size'])

        files.append({
            'filename':filename.encode(encoding='ascii',errors='ignore').decode(),
            'extension':ext.encode(encoding='ascii',errors='ignore').decode(),
            'size':k.next_sibling.next_sibling.contents[0].encode(encoding='ascii',errors='ignore').decode(),
            'contents':archive,
            'nfile_archive':archive.shape[0]
            })

       
    return pd.DataFrame.from_dict(files)


In [4]:
date='20190827'
zenododata=pd.read_pickle("processed_data/" + date + "/zenododata2.pkl",compression='gzip')

In [10]:
dataset=zenododata[zenododata.category=='dataset']

In [6]:
dataset

,category,date,title,authors,abstract,keywords,license,url,files
3,dataset,2017-03-03,InSAR 2015 Nepal EQ Sentinel 1 ESA,[Antoine Lucas],InSAR for the 215 Nepal EQ using Sentinel 1 Da...,"[InSAR, Sentinel-1, Nepal, 2015 EQ]",http://creativecommons.org/licenses/by/4.0/leg...,https://zenodo.org/record/345302,NaN
42,dataset,2017-07-28,Integrative assessment of climate change-relat...,"[Scheuer, Sebastian]",Shapefile data set estimating trends of mean a...,"[climate change, urban land development, clima...",http://creativecommons.org/licenses/by/4.0/leg...,https://zenodo.org/record/835650,NaN
80,dataset,2018-05-01,Binary black-hole simulation SXS:BBH:0934,[SXS Collaboration],Simulation of a black-hole binary system evolv...,[],http://creativecommons.org/publicdomain/zero/1...,https://zenodo.org/record/2622615,NaN
84,dataset,2016-06-07,PowerTAC 2016-06 Finals Game 23,[PowerTAC],Log and boot files of game 23,[PowerTAC],http://creativecommons.org/licenses/by/4.0/leg...,https://zenodo.org/record/582937,NaN
106,dataset,2018-05-01,Binary black-hole simulation SXS:BBH:1073,[SXS Collaboration],Simulation of a black-hole binary system evolv...,[],http://creativecommons.org/publicdomain/zero/1...,https://zenodo.org/record/2623887,NaN
128,dataset,2017-05-01,This is the title for testFolder1,[NP],no_description_available,[],None,https://zenodo.org/record/570233,NaN
130,dataset,2017-10-20,RB559 srp-8(ok291)V | 2011-09-20T12:41:00+01:00,"[Javer, Avelino, Currie, Michael, Hokanson, Ji...",\nThis experiment is part of the C.elegans beh...,[],http://creativecommons.org/licenses/by/4.0/leg...,https://zenodo.org/record/1029133,NaN
136,dataset,2017-10-20,NL332 gpa-1(pk15)V | 2009-06-25T15:00:00+01:00,"[Javer, Avelino, Currie, Michael, Hokanson, Ji...",\nThis experiment is part of the C.elegans beh...,[],http://creativecommons.org/licenses/by/4.0/leg...,https://zenodo.org/record/1032547,NaN
165,dataset,2017-10-20,"N2 Schafer Lab N2 (Bristol, UK) | 2010-11-11T1...","[Javer, Avelino, Currie, Michael, Hokanson, Ji...",\nThis experiment is part of the C.elegans beh...,[],http://creativecommons.org/licenses/by/4.0/leg...,https://zenodo.org/record/1008008,NaN
176,dataset,2019-07-11,Binary black-hole simulation SXS:BBH:0395,[SXS Collaboration],Simulation of a black-hole binary system evolv...,[],http://creativecommons.org/licenses/by/4.0/leg...,https://zenodo.org/record/3317185,NaN


In [11]:
subdataset=dataset[1:100]
subdataset=subdataset.drop(['files'], axis=1)
subdataset['files']=subdataset.apply(lambda x: getarchives(x['url']), axis=1)

https://zenodo.org/record/835650
https://zenodo.org/record/835650
https://zenodo.org/record/2622615
https://zenodo.org/record/582937
https://zenodo.org/record/2623887
https://zenodo.org/record/570233
https://zenodo.org/record/1029133
https://zenodo.org/record/1032547
https://zenodo.org/record/1008008
https://zenodo.org/record/3317185
https://zenodo.org/record/901425
https://zenodo.org/record/1027930
https://zenodo.org/record/3263930
https://zenodo.org/record/2578453
https://zenodo.org/record/842005
https://zenodo.org/record/2626109
https://zenodo.org/record/3227862
https://zenodo.org/record/1345172
https://zenodo.org/record/3340879
https://zenodo.org/record/902972
https://zenodo.org/record/3273047
https://zenodo.org/record/1007699
https://zenodo.org/record/439582
https://zenodo.org/record/3312485
https://zenodo.org/record/1288500
https://zenodo.org/record/61269
https://zenodo.org/record/855339
https://zenodo.org/record/1481929
https://zenodo.org/record/1021167
https://zenodo.org/record

In [12]:
subdataset

,category,date,title,authors,abstract,keywords,license,url,files
42,dataset,2017-07-28,Integrative assessment of climate change-relat...,"[Scheuer, Sebastian]",Shapefile data set estimating trends of mean a...,"[climate change, urban land development, clima...",http://creativecommons.org/licenses/by/4.0/leg...,https://zenodo.org/record/835650,co...
80,dataset,2018-05-01,Binary black-hole simulation SXS:BBH:0934,[SXS Collaboration],Simulation of a black-hole binary system evolv...,[],http://creativecommons.org/publicdomain/zero/1...,https://zenodo.org/record/2622615,c...
84,dataset,2016-06-07,PowerTAC 2016-06 Finals Game 23,[PowerTAC],Log and boot files of game 23,[PowerTAC],http://creativecommons.org/licenses/by/4.0/leg...,https://zenodo.org/record/582937,co...
106,dataset,2018-05-01,Binary black-hole simulation SXS:BBH:1073,[SXS Collaboration],Simulation of a black-hole binary system evolv...,[],http://creativecommons.org/publicdomain/zero/1...,https://zenodo.org/record/2623887,c...
128,dataset,2017-05-01,This is the title for testFolder1,[NP],no_description_available,[],None,https://zenodo.org/record/570233,Empty DataFrame Columns: [] Index: []
130,dataset,2017-10-20,RB559 srp-8(ok291)V | 2011-09-20T12:41:00+01:00,"[Javer, Avelino, Currie, Michael, Hokanson, Ji...",\nThis experiment is part of the C.elegans beh...,[],http://creativecommons.org/licenses/by/4.0/leg...,https://zenodo.org/record/1029133,co...
136,dataset,2017-10-20,NL332 gpa-1(pk15)V | 2009-06-25T15:00:00+01:00,"[Javer, Avelino, Currie, Michael, Hokanson, Ji...",\nThis experiment is part of the C.elegans beh...,[],http://creativecommons.org/licenses/by/4.0/leg...,https://zenodo.org/record/1032547,co...
165,dataset,2017-10-20,"N2 Schafer Lab N2 (Bristol, UK) | 2010-11-11T1...","[Javer, Avelino, Currie, Michael, Hokanson, Ji...",\nThis experiment is part of the C.elegans beh...,[],http://creativecommons.org/licenses/by/4.0/leg...,https://zenodo.org/record/1008008,co...
176,dataset,2019-07-11,Binary black-hole simulation SXS:BBH:0395,[SXS Collaboration],Simulation of a black-hole binary system evolv...,[],http://creativecommons.org/licenses/by/4.0/leg...,https://zenodo.org/record/3317185,c...
199,dataset,2013-09-16,Supplementary material 5 from: Wayland M (2013...,"[Wayland, Matthew]",Authors: Matthew T Wayland\n Data t...,"[Acanthocephala, Echinorhynchus truttae, Echin...",http://creativecommons.org/licenses/by/4.0/leg...,https://zenodo.org/record/901425,co...


In [29]:
n=subdataset[subdataset.url=="https://zenodo.org/record/842005"].files

In [30]:
n.shape

(1,)

In [23]:
getarchives("https://zenodo.org/record/1460470")

https://zenodo.org/record/1460470

<!DOCTYPE html>

<html dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="5fPGCLllnWrvFxH9QWI0l1TadV7byeEvfPcyK2VkS_s" name="google-site-verification">
<meta content="Rp5zp04IKW-s1IbpTOGB7Z6XY60oloZD5C3kTM-AiY4" name="google-site-verification">
<meta content="umenay8zh4kswbi568zqp19bqb-jvngusibub1ygib0x3jne9rig0fnmtofm8abb7lkzgltqp5yhm68s5qz4iqqkm39xl2o-p5foixd-1xfq4yig07ugcd1sp5kmyvpe" name="norton-safeweb-site-verification">
<title>sardinia_tasmax_GFDL-ESM2M_rcp2p6_2006-2099_monthly.dat | Zenodo</title>
<link href="/static/favicon.ico" rel="shortcut icon"/>
<link href="/static/apple-touch-icon-144-precomposed.png" rel="apple-touch-icon-precomposed" sizes="144x144"/>
<link href="/static/apple-touch-icon-114-precomposed.png" rel="apple-touch-icon-precomposed" sizes="114x114"/>
<link href="/static/apple-touch-icon

,contents,extension,filename,nfile_archive,size
0,"Empty DataFrame Columns: [filename, extension,...",.dat,sardinia_tasmax_GFDL-ESM2M_rcp2p6_2006-2099_mo...,0,890.0 kB


In [32]:
r= requests.get("https://zenodo.org/api/records/1460470")

In [35]:
resp=r.json()

In [38]:
resp['files'][0]['type']

'dat'

In [40]:
"https://zenodo.org/record/1460470".split("/")[-1]

'1460470'